# Estandarizacion de datos de los Anuarios Geoestadísticos de INEGI 2017

## 1. Introduccion
Parámetros que salen de esta fuente:

ID |Descripción
---|:----------
P0610|Ventas de electricidad
P0701|Longitud total de la red de carreteras del municipio (excluyendo las autopistas)

In [1]:
descripciones = {
    'P0610': 'Ventas de electricidad',
    'P0701': 'Longitud total de la red de carreteras del municipio (excluyendo las autopistas)'
}

In [13]:
# Librerias utilizadas
import pandas as pd
import sys
import urllib
import os
import csv
import zipfile

In [3]:
# Configuracion del sistema
print('Python {} on {}'.format(sys.version, sys.platform))
print('Pandas version: {}'.format(pd.__version__))
import platform; print('Running on {} {}'.format(platform.system(), platform.release()))

Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)] on win32
Pandas version: 0.20.1
Running on Windows 8.1


## 2. Descarga de datos
Cada entidad cuenta con una página que presenta sus respectivos anuarios geoestadísticos. La manera más rápida de obtener las ligas de los anuarios es entrar a la biblioteca de INEGI (http://www.beta.inegi.org.mx/app/publicaciones/) y buscar la palabra "Anuario" en el campo de búsqueda.

CVE_EDO |Nombre|URL
---|:---|:----------
01|Aguascalientes|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092078
02|Baja California|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094874
03|Baja California Sur|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094881
04|Campeche|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825095109
05|Coahuila de Zaragoza|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825095406
06|Colima|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092061
07|Chiapas|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094836
08|Chihuahua|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092139
09|Ciudad de México|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094683
10|Durango|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092115
11|Guanajuato|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092146
12|Guerrero|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094690
13|Hidalgo|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825095093
14|Jalisco|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092085
15|México|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094706
16|Michoacán de Ocampo|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092092
17|Morelos|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094713
18|Nayarit|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092054
19|Nuevo León|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094911
20|Oaxaca|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094843
21|Puebla|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094973
22|Querétaro|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092108
23|Quintana Roo|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825095130
24|San Luis Potosí|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092122
25|Sinaloa|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094898
26|Sonora|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094904
27|Tabasco|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825095123
28|Tamaulipas|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094928
29|Tlaxcala|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825096212
30|Veracruz de Ignacio de la Llave|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825094980
31|Yucatán|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825095116
32|Zacatecas|http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825092047

Dentro de cada página, se incluye una liga directa para descargar un archivo comprimido con las tablas de datos de cada anuario geoestadítico. La lista links contiene estas URL y se utilizará para sistematizar la descarga de datos.

In [9]:
raiz = 'http://internet.contenidos.inegi.org.mx/contenidos/Productos/prod_serv/contenidos/espanol/bvinegi/productos/nueva_estruc/anuarios_2017/'
# El diccionario tiene como llave la CVE_EDO y dirige hacia la liga de descarga del archivo zip con las tablas del 
# Anuario Geoestadístico de cada estado
links = {
    '01': raiz + '702825092078.zip',
    '02': raiz + '702825094874.zip',
    '03': raiz + '702825094881.zip',
    '04': raiz + '702825095109.zip',
    '05': raiz + '702825095406.zip',
    '06': raiz + '702825092061.zip',
    '07': raiz + '702825094836.zip',
    '08': raiz + '702825092139.zip',
    '09': raiz + '702825094683.zip',
    '10': raiz + '702825092115.zip',
    '11': raiz + '702825092146.zip',
    '12': raiz + '702825094690.zip',
    '13': raiz + '702825095093.zip',
    '14': raiz + '702825092085.zip',
    '15': raiz + '702825094706.zip',
    '16': raiz + '702825092092.zip',
    '17': raiz + '702825094713.zip',
    '18': raiz + '702825092054.zip',
    '19': raiz + '702825094911.zip',
    '20': raiz + '702825094843.zip',
    '21': raiz + '702825094973.zip',
    '22': raiz + '702825092108.zip',
    '23': raiz + '702825095130.zip',
    '24': raiz + '702825092122.zip',
    '25': raiz + '702825094898.zip',
    '26': raiz + '702825094904.zip',
    '27': raiz + '702825095123.zip',
    '28': raiz + '702825094928.zip',
    '29': raiz + '702825096212.zip',
    '30': raiz + '702825094980.zip',
    '31': raiz + '702825095116.zip',
    '32': raiz + '702825092047.zip'
}

In [10]:
# Descarga de archivos a carpeta local
destino = r'D:\PCCS\00_RawData\01_CSV\AGEO\2017'
archivos = {}   # Diccionario para guardar memoria de descarga
for k,v in links.items():
    archivo_local = destino + r'\{}.zip'.format(k)
    if os.path.isfile(archivo_local):
        print('Ya existe el archivo: {}'.format(archivo_local))
        archivos[k] = archivo_local
    else:
        print('Descargando {} ... ... ... ... ... '.format(archivo_local))
        urllib.request.urlretrieve(v, archivo_local) #
        archivos[k] = archivo_local
        print('se descargó {}'.format(archivo_local))

Descargando D:\PCCS\00_RawData\01_CSV\AGEO\2017\01.zip ... ... ... ... ... 
se descargó D:\PCCS\00_RawData\01_CSV\AGEO\2017\01.zip
Descargando D:\PCCS\00_RawData\01_CSV\AGEO\2017\02.zip ... ... ... ... ... 
se descargó D:\PCCS\00_RawData\01_CSV\AGEO\2017\02.zip
Descargando D:\PCCS\00_RawData\01_CSV\AGEO\2017\03.zip ... ... ... ... ... 
se descargó D:\PCCS\00_RawData\01_CSV\AGEO\2017\03.zip
Descargando D:\PCCS\00_RawData\01_CSV\AGEO\2017\04.zip ... ... ... ... ... 
se descargó D:\PCCS\00_RawData\01_CSV\AGEO\2017\04.zip
Descargando D:\PCCS\00_RawData\01_CSV\AGEO\2017\05.zip ... ... ... ... ... 
se descargó D:\PCCS\00_RawData\01_CSV\AGEO\2017\05.zip
Descargando D:\PCCS\00_RawData\01_CSV\AGEO\2017\06.zip ... ... ... ... ... 
se descargó D:\PCCS\00_RawData\01_CSV\AGEO\2017\06.zip
Descargando D:\PCCS\00_RawData\01_CSV\AGEO\2017\07.zip ... ... ... ... ... 
se descargó D:\PCCS\00_RawData\01_CSV\AGEO\2017\07.zip
Descargando D:\PCCS\00_RawData\01_CSV\AGEO\2017\08.zip ... ... ... ... ... 
se desc

In [14]:
# Descompresión de archivos de estado
unzipped = {}
for estado, comprimido in archivos.items():
    target = destino + '\\' + estado
    if os.path.isdir(target):
        print('Ya existe el directorio: {}'.format(target))
        unzipped[estado] = target
    else:
        print('Descomprimiendo {} ... ... ... ... ... '.format(target))
        descomprimir = zipfile.ZipFile(comprimido, 'r')
        descomprimir.extractall(target)
        descomprimir.close
        unzipped[estado] = target
        

Descomprimiendo D:\PCCS\00_RawData\01_CSV\AGEO\2017\01 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\AGEO\2017\02 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\AGEO\2017\03 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\AGEO\2017\04 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\AGEO\2017\05 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\AGEO\2017\06 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\AGEO\2017\07 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\AGEO\2017\08 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\AGEO\2017\09 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\AGEO\2017\10 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\AGEO\2017\11 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\AGEO\2017\12 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\AGEO\2017\13 ... ... ... ... ... 
Descomprimie

In [21]:
os.listdir(unzipped['01'])[1]

True

### Extraccion de indices
Conocer la información que contiene cada hoja del índice geoestadístico puede ser muy valioso y es necesario para hacer una función que itere adecuadamente por los archivos de todos los estados, debido a que los anuarios geoestadísticos de cada estado tienen ligeras variaciones que impiden una iteración directa.

In [19]:
unzipped

{'01': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\01',
 '02': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\02',
 '03': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\03',
 '04': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\04',
 '05': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\05',
 '06': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\06',
 '07': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\07',
 '08': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\08',
 '09': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\09',
 '10': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\10',
 '11': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\11',
 '12': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\12',
 '13': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\13',
 '14': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\14',
 '15': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\15',
 '16': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\16',
 '17': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\17',
 '18': 'D:\\PCCS\\00_RawData\\01_CSV\\AGEO\\2017\\18',
 '19': 'D:

In [106]:
list(test)

['1. Aspectos geográficos',
 'Unnamed: 1',
 'Unnamed: 2',
 'Unnamed: 3',
 'Unnamed: 4',
 'Unnamed: 5',
 'Unnamed: 6',
 'Unnamed: 7',
 'Unnamed: 8',
 'indice']

In [107]:
# Extraer indices
indices = {}
for estado, ruta in unzipped.items():
    for file in os.listdir(ruta):
        if file.endswith('.xls'):
            path = ruta + '\\' + file
            indice = pd.read_excel(path, sheetname='Índice', skiprows = 1)   # Primera lectura al indice para sacar columnas
            dtypes = list(indice)
            tempdic = {}
            for i in dtypes:
                tempdic[i] = 'str'
            indice = pd.read_excel(path, 
                                   sheetname='Índice', 
                                   skiprows = 1,
                                   dtype = tempdic).dropna(how = 'all')   # Segunda lectura al indice ya con dtypes
            name = list(indice)[0]           # Guarda el nombre del indice
            cols = []
            for i in range(len(list(indice))):
                cols.append('col{}'.format(i))    # Crea nombres estandar de columna
            indice.columns = cols                 # Asigna nombres de columna
            indice['indice'] = name
            indice['file'] = file
            if estado not in indices.keys():      # Crea un diccionario para cada estado, si no existe
                indices[estado] = {}
            indices[estado][name] = indice
            print('Procesado {} |||NOMBRE:||| {}; [{}]'.format(file, name, len(cols)))     # Imprime los resultados del proceso

Procesado Ags. 1 Aspectos geográficos.xls |||NOMBRE:||| 1. Aspectos geográficos; [9]
Procesado Ags. 10 Trabajo.xls |||NOMBRE:||| 10. Trabajo; [3]
Procesado Ags. 11 Información económica agregada.xls |||NOMBRE:||| 11. Información económica agregada; [3]
Procesado Ags. 12 Agricultura.xls |||NOMBRE:||| 12. Agricultura; [3]
Procesado Ags. 13 Ganadería.xls |||NOMBRE:||| 13. Ganadería; [3]
Procesado Ags. 14 Aprovechamiento forestal.xls |||NOMBRE:||| 14. Aprovechamiento forestal; [3]
Procesado Ags. 15 Pesca.xls |||NOMBRE:||| 15. Pesca; [3]
Procesado Ags. 16 Minería.xls |||NOMBRE:||| 16. Minería; [3]
Procesado Ags. 17 Industria manufacturera.xls |||NOMBRE:||| 17. Industria manufacturera; [3]
Procesado Ags. 18 Construcción.xls |||NOMBRE:||| 18. Construcción; [3]
Procesado Ags. 19 Electricidad.xls |||NOMBRE:||| 19. Electricidad; [3]
Procesado Ags. 2 Medio ambiente.xls |||NOMBRE:||| 2. Medio ambiente; [3]
Procesado Ags. 20 Comercio.xls |||NOMBRE:||| 20. Comercio; [3]
Procesado Ags. 21 Turismo.xls

Procesado Coah. 19 Electricidad.xls |||NOMBRE:||| 19. Electricidad; [3]
Procesado Coah. 2 Medio ambiente.xls |||NOMBRE:||| 2. Medio ambiente; [3]
Procesado Coah. 20 Comercio.xls |||NOMBRE:||| 20. Comercio; [3]
Procesado Coah. 21 Turismo.xls |||NOMBRE:||| 21. Turismo; [3]
Procesado Coah. 22 Transportes y comunicaciones.xls |||NOMBRE:||| 22. Transportes y comunicaciones; [3]
Procesado Coah. 23 Indicadores financieros.xls |||NOMBRE:||| 23. Indicadores financieros; [3]
Procesado Coah. 24 Finanzas públicas.xls |||NOMBRE:||| 24. Finanzas públicas; [3]
Procesado Coah. 25 Sector externo.xls |||NOMBRE:||| 25. Sector externo; [3]
Procesado Coah. 3 Población.xls |||NOMBRE:||| 3. Población; [3]
Procesado Coah. 4 Vivienda y urbanización.xls |||NOMBRE:||| 4. Vivienda y urbanización; [3]
Procesado Coah. 5 Salud.xls |||NOMBRE:||| 5. Salud; [3]
Procesado Coah. 6 Educación, ciencia y tecnología.xls |||NOMBRE:||| 6. Educación, ciencia y tecnología; [3]
Procesado Coah. 7 Cultura y deporte.xls |||NOMBRE:||

Procesado CDMX. 3 Población.xls |||NOMBRE:||| 3. Población; [3]
Procesado CDMX. 4 Vivienda y urbanización.xls |||NOMBRE:||| 4. Vivienda y urbanización; [3]
Procesado CDMX. 5 Salud.xls |||NOMBRE:||| 5. Salud; [3]
Procesado CDMX. 6 Educación, ciencia y tecnología.xls |||NOMBRE:||| 6. Educación, ciencia y tecnología; [3]
Procesado CDMX. 7 Cultura y deporte.xls |||NOMBRE:||| 7. Cultura y deporte; [3]
Procesado CDMX. 8 Gobierno.xls |||NOMBRE:||| 8. Gobierno; [3]
Procesado CDMX. 9 Seguridad y justicia.xls |||NOMBRE:||| 9. Seguridad y justicia; [3]
Procesado Dgo. 1 Aspectos geográficos.xls |||NOMBRE:||| 1. Aspectos geográficos; [9]
Procesado Dgo. 10 Trabajo.xls |||NOMBRE:||| 10. Trabajo; [3]
Procesado Dgo. 11 Información económica agregada.xls |||NOMBRE:||| 11. Información económica agregada; [3]
Procesado Dgo. 12 Agricultura.xls |||NOMBRE:||| 12. Agricultura; [3]
Procesado Dgo. 13 Ganadería.xls |||NOMBRE:||| 13. Ganadería; [3]
Procesado Dgo. 14 Aprovechamiento forestal.xls |||NOMBRE:||| 14. 

Procesado Jal. 1 Aspectos geográficos.xls |||NOMBRE:||| 1. Aspectos geográficos; [9]
Procesado Jal. 10 Trabajo.xls |||NOMBRE:||| 10. Trabajo; [3]
Procesado Jal. 11 Información económica agregada.xls |||NOMBRE:||| 11. Información económica agregada; [3]
Procesado Jal. 12 Agricultura.xls |||NOMBRE:||| 12. Agricultura; [3]
Procesado Jal. 13 Ganadería.xls |||NOMBRE:||| 13. Ganadería; [3]
Procesado Jal. 14 Aprovechamiento forestal.xls |||NOMBRE:||| 14. Aprovechamiento forestal; [3]
Procesado Jal. 15 Pesca.xls |||NOMBRE:||| 15. Pesca; [3]
Procesado Jal. 16 Minería.xls |||NOMBRE:||| 16. Minería; [3]
Procesado Jal. 17 Industria manufacturera.xls |||NOMBRE:||| 17. Industria manufacturera; [3]
Procesado Jal. 18 Construcción.xls |||NOMBRE:||| 18. Construcción; [3]
Procesado Jal. 19 Electricidad.xls |||NOMBRE:||| 19. Electricidad; [3]
Procesado Jal. 2 Medio ambiente.xls |||NOMBRE:||| 2. Medio ambiente; [3]
Procesado Jal. 20 Comercio.xls |||NOMBRE:||| 20. Comercio; [3]
Procesado Jal. 21 Turismo.xls

Procesado Nay. 17 Industria manufacturera.xls |||NOMBRE:||| 17. Industria manufacturera; [3]
Procesado Nay. 18 Construcción.xls |||NOMBRE:||| 18. Construcción; [3]
Procesado Nay. 19 Electricidad.xls |||NOMBRE:||| 19. Electricidad; [3]
Procesado Nay. 2 Medio ambiente.xls |||NOMBRE:||| 2. Medio ambiente; [3]
Procesado Nay. 20 Comercio.xls |||NOMBRE:||| 20. Comercio; [3]
Procesado Nay. 21 Turismo.xls |||NOMBRE:||| 21. Turismo; [3]
Procesado Nay. 22 Transportes y comunicaciones.xls |||NOMBRE:||| 22. Transportes y comunicaciones; [3]
Procesado Nay. 23 Indicadores financieros.xls |||NOMBRE:||| 23. Indicadores financieros; [3]
Procesado Nay. 24 Finanzas públicas.xls |||NOMBRE:||| 24. Finanzas públicas; [3]
Procesado Nay. 25 Sector externo.xls |||NOMBRE:||| 25. Sector externo; [3]
Procesado Nay. 3 Población.xls |||NOMBRE:||| 3. Población; [3]
Procesado Nay. 4 Vivienda y urbanización.xls |||NOMBRE:||| 4. Vivienda y urbanización; [3]
Procesado Nay. 5 Salud.xls |||NOMBRE:||| 5. Salud; [3]
Procesa

Procesado Qro. 24 Finanzas públicas.xls |||NOMBRE:||| 24. Finanzas públicas; [3]
Procesado Qro. 25 Sector externo.xls |||NOMBRE:||| 25. Sector externo; [3]
Procesado Qro. 3 Población.xls |||NOMBRE:||| 3. Población; [3]
Procesado Qro. 4 Vivienda y urbanización.xls |||NOMBRE:||| 4. Vivienda y urbanización; [3]
Procesado Qro. 5 Salud.xls |||NOMBRE:||| 5. Salud; [3]
Procesado Qro. 6 Educación, ciencia y tecnología.xls |||NOMBRE:||| 6. Educación, ciencia y tecnología; [3]
Procesado Qro. 7 Cultura y deporte.xls |||NOMBRE:||| 7. Cultura y deporte; [3]
Procesado Qro. 8 Gobierno.xls |||NOMBRE:||| 8. Gobierno; [3]
Procesado Qro. 9 Seguridad y justicia.xls |||NOMBRE:||| 9. Seguridad y justicia; [3]
Procesado Qroo. 1 Aspectos geográficos.xls |||NOMBRE:||| 1. Aspectos geográficos; [9]
Procesado Qroo. 10 Trabajo.xls |||NOMBRE:||| 10. Trabajo; [3]
Procesado Qroo. 11 Información económica agregada.xls |||NOMBRE:||| 11. Información económica agregada; [3]
Procesado Qroo. 12 Agricultura.xls |||NOMBRE:||

Procesado Tab. 1 Aspectos geográficos.xls |||NOMBRE:||| 1. Aspectos geográficos; [9]
Procesado Tab. 10 Trabajo.xls |||NOMBRE:||| 10. Trabajo; [3]
Procesado Tab. 11 Información económica agregada.xls |||NOMBRE:||| 11. Información económica agregada; [3]
Procesado Tab. 12 Agricultura.xls |||NOMBRE:||| 12. Agricultura; [3]
Procesado Tab. 13 Ganadería.xls |||NOMBRE:||| 13. Ganadería; [3]
Procesado Tab. 14 Aprovechamiento forestal.xls |||NOMBRE:||| 14. Aprovechamiento forestal; [3]
Procesado Tab. 15 Pesca.xls |||NOMBRE:||| 15. Pesca; [3]
Procesado Tab. 16 Minería.xls |||NOMBRE:||| 16. Minería; [3]
Procesado Tab. 17 Industria manufacturera.xls |||NOMBRE:||| 17. Industria manufacturera; [3]
Procesado Tab. 18 Construcción.xls |||NOMBRE:||| 18. Construcción; [3]
Procesado Tab. 19 Electricidad.xls |||NOMBRE:||| 19. Electricidad; [3]
Procesado Tab. 2 Medio ambiente.xls |||NOMBRE:||| 2. Medio ambiente; [3]
Procesado Tab. 20 Comercio.xls |||NOMBRE:||| 20. Comercio; [3]
Procesado Tab. 21 Turismo.xls

Procesado Yuc. 14 Aprovechamiento forestal.xls |||NOMBRE:||| 14. Aprovechamiento forestal; [3]
Procesado Yuc. 15 Pesca.xls |||NOMBRE:||| 15. Pesca; [3]
Procesado Yuc. 16 Minería.xls |||NOMBRE:||| 16. Minería; [3]
Procesado Yuc. 17 Industria manufacturera.xls |||NOMBRE:||| 17. Industria manufacturera; [3]
Procesado Yuc. 18 Construcción.xls |||NOMBRE:||| 18. Construcción; [3]
Procesado Yuc. 19 Electricidad.xls |||NOMBRE:||| 19. Electricidad; [3]
Procesado Yuc. 2 Medio ambiente.xls |||NOMBRE:||| 2. Medio ambiente; [3]
Procesado Yuc. 20 Comercio.xls |||NOMBRE:||| 20. Comercio; [3]
Procesado Yuc. 21 Turismo.xls |||NOMBRE:||| 21. Turismo; [3]
Procesado Yuc. 22 Transportes y comunicaciones.xls |||NOMBRE:||| 22. Transportes y comunicaciones; [3]
Procesado Yuc. 23 Indicadores financieros.xls |||NOMBRE:||| 23. Indicadores financieros; [3]
Procesado Yuc. 24 Finanzas públicas.xls |||NOMBRE:||| 24. Finanzas públicas; [3]
Procesado Yuc. 25 Sector externo.xls |||NOMBRE:||| 25. Sector externo; [3]
Pro

In [108]:
# Reordenar los dataframes por tipo
indices_2 = {}
for estado in indices.keys():
    for indice in indices[estado].keys():
        if indice not in indices_2.keys():
            indices_2[indice] = {}
        indices_2[indice][estado] = indices[estado][indice]

In [109]:
# Convertir indices en archivos unicos.
finalindexes = {}
for i in indices_2.keys():
    print(i)
    frameslist = []
    for estado in indices_2[i].keys():
        frame = indices_2[i][estado]
        frame['estado'] = estado
        frameslist.append(frame)
    fullindex = pd.concat(frameslist)
    finalindexes[i] = fullindex
    print('Hecho: {}\n'.format(i))

1. Aspectos geográficos
Hecho: 1. Aspectos geográficos

10. Trabajo
Hecho: 10. Trabajo

11. Información económica agregada
Hecho: 11. Información económica agregada

12. Agricultura
Hecho: 12. Agricultura

13. Ganadería
Hecho: 13. Ganadería

14. Aprovechamiento forestal
Hecho: 14. Aprovechamiento forestal

15. Pesca
Hecho: 15. Pesca

16. Minería
Hecho: 16. Minería

17. Industria manufacturera
Hecho: 17. Industria manufacturera

18. Construcción
Hecho: 18. Construcción

19. Electricidad
Hecho: 19. Electricidad

2. Medio ambiente
Hecho: 2. Medio ambiente

20. Comercio
Hecho: 20. Comercio

21. Turismo
Hecho: 21. Turismo

22. Transportes y comunicaciones
Hecho: 22. Transportes y comunicaciones

23. Indicadores financieros
Hecho: 23. Indicadores financieros

24. Finanzas públicas
Hecho: 24. Finanzas públicas

25. Sector externo
Hecho: 25. Sector externo

3. Población
Hecho: 3. Población

4. Vivienda y urbanización
Hecho: 4. Vivienda y urbanización

5. Salud
Hecho: 5. Salud

6. Educación, ci

In [110]:
# Escribir archivos xlsx
path = r'D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices'
for indice in finalindexes.keys():
    file = path+'\\'+indice+'.xlsx'
    writer = pd.ExcelWriter(file)
    finalindexes[indice].to_excel(writer, sheet_name = 'Indice')
    writer.save()
    print('[{}] lineas - archivo {}'.format(len(finalindexes[indice]), file))

[1852] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\1. Aspectos geográficos.xlsx
[3343] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\10. Trabajo.xlsx
[896] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\11. Información económica agregada.xlsx
[540] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\12. Agricultura.xlsx
[1164] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\13. Ganadería.xlsx
[966] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\14. Aprovechamiento forestal.xlsx
[269] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\15. Pesca.xlsx
[275] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\16. Minería.xlsx
[169] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\17. Industria manufacturera.xlsx
[593] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\18. Construcción.xlsx
[822] lineas - archivo D:\PCCS\01_Dmine\Datasets\AGEO\2017\indices\19. Electrici

Los índices obtenidos de esta manera recibirán una limpieza manual desde Excel.